In [1]:
import pandas as pd
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
# 파이썬 구글 빅쿼리 연동 코드
# json 파일

key_path = glob.glob("./*.json")[0]
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [3]:
def sql_to_dataframe(sql:str) -> pd.DataFrame:
    """
    Args:
        sql (str): sql for extraction

    Returns:
        pd.DataFrame: extract data with sql
    """
    query_job = client.query(sql)
    df = query_job.to_dataframe()
    return df

### 시간별 신규 사용자

In [4]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  user.event_date,
  CONCAT(user.traffic_source.source,"/",user.traffic_source.medium) AS source_medium,
  COUNT(DISTINCT user.user_pseudo_id) AS new_user_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY user.event_date, source_medium
"""

df = sql_to_dataframe(sql)
df

,event_date,source_medium,new_user_count
0,20201116,google/cpc,131
1,20201116,(direct)/(none),663
2,20201116,<Other>/referral,252
3,20201116,google/organic,887
4,20201116,<Other>/<Other>,492
...,...,...,...
235,20201122,<Other>/(data deleted),2
236,20201206,<Other>/(data deleted),1
237,20201121,<Other>/(data deleted),3
238,20201124,<Other>/(data deleted),1


### 새 사용자

In [5]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(DISTINCT user_pseudo_id) AS new_user_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY new_user_count DESC
"""

df = sql_to_dataframe(sql)
df

,source_medium,new_user_count
0,google/organic,26937
1,(direct)/(none),19504
2,<Other>/<Other>,13380
3,<Other>/referral,8197
4,shop.googlemerchandisestore.com/referral,6256
5,google/cpc,4106
6,(data deleted)/(data deleted),3856
7,<Other>/organic,2560
8,<Other>/(data deleted),77


### 참여 세션수

In [6]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS engaged_session_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
AND (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "session_engaged") = 1
GROUP BY source_medium
ORDER BY engaged_session_count DESC"""

df = sql_to_dataframe(sql)
df


,source_medium,engaged_session_count
0,google/organic,26401
1,(direct)/(none),18427
2,<Other>/<Other>,12991
3,<Other>/referral,7351
4,shop.googlemerchandisestore.com/referral,4851
5,google/cpc,3958
6,<Other>/organic,2386
7,(data deleted)/(data deleted),1951
8,<Other>/(data deleted),34


### 세션수

In [7]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS session_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY session_count DESC"""

df = sql_to_dataframe(sql)
df

,source_medium,session_count
0,google/organic,29006
1,(direct)/(none),21065
2,<Other>/<Other>,13648
3,<Other>/referral,8554
4,shop.googlemerchandisestore.com/referral,6742
5,(data deleted)/(data deleted),4533
6,google/cpc,4134
7,<Other>/organic,2573
8,<Other>/(data deleted),79


### 참여율

In [8]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS session_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY session_count DESC"""

df = sql_to_dataframe(sql)
df

,source_medium,session_count
0,google/organic,29006
1,(direct)/(none),21065
2,<Other>/<Other>,13648
3,<Other>/referral,8554
4,shop.googlemerchandisestore.com/referral,6742
5,(data deleted)/(data deleted),4533
6,google/cpc,4134
7,<Other>/organic,2573
8,<Other>/(data deleted),79


### 사용자당 참여 세션수

In [9]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  engaged_session.source_medium,
  engaged_session.engaged_session_count,
  session.new_user_count,
  ROUND(engaged_session.engaged_session_count / session.new_user_count,2) AS user_per_engagement_count
FROM(
  SELECT
    CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
    COUNT(DISTINCT user_pseudo_id) AS new_user_count
  FROM user
  WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
  GROUP BY source_medium
) session
INNER JOIN(
  SELECT
    CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
    COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS engaged_session_count
  FROM user
  WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
  AND (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "session_engaged") = 1
  GROUP BY source_medium
) engaged_session
ON session.source_medium = engaged_session.source_medium
"""

df = sql_to_dataframe(sql)
df

,source_medium,engaged_session_count,new_user_count,user_per_engagement_count
0,shop.googlemerchandisestore.com/referral,4851,6256,0.78
1,(direct)/(none),18427,19504,0.94
2,<Other>/<Other>,12991,13380,0.97
3,google/organic,26401,26937,0.98
4,<Other>/referral,7351,8197,0.90
5,google/cpc,3958,4106,0.96
6,(data deleted)/(data deleted),1951,3856,0.51
7,<Other>/organic,2386,2560,0.93
8,<Other>/(data deleted),34,77,0.44


### 평균 참여 시간

In [10]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  FLOOR(SUM(FLOOR((SELECT value.int_value FROM UNNEST(event_params) WHERE key = "engagement_time_msec") / 1000)) / COUNT(DISTINCT user_pseudo_id)) AS engagement_time
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY engagement_time DESC
"""

df = sql_to_dataframe(sql)
df

,source_medium,engagement_time
0,(data deleted)/(data deleted),110.0
1,<Other>/referral,84.0
2,google/organic,84.0
3,shop.googlemerchandisestore.com/referral,84.0
4,(direct)/(none),83.0
5,<Other>/(data deleted),82.0
6,<Other>/<Other>,81.0
7,google/cpc,78.0
8,<Other>/organic,77.0


### 이벤트수

In [11]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(event_name) AS event_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY event_count DESC"""

df = sql_to_dataframe(sql)
df

,source_medium,event_count
0,google/organic,396733
1,(direct)/(none),275795
2,<Other>/<Other>,186686
3,<Other>/referral,114575
4,shop.googlemerchandisestore.com/referral,86619
5,google/cpc,55833
6,(data deleted)/(data deleted),52113
7,<Other>/organic,33801
8,<Other>/(data deleted),967


### 전환

In [12]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  COUNT(event_name) AS conversion_count
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
AND event_name IN("purchase","begin_checkout","first_visit","predict_ltv_payer","add_to_cart","view_item","view_cart","qualified_visit","Membership","predicted_top_spenders")
GROUP BY source_medium
ORDER BY conversion_count DESC"""

df = sql_to_dataframe(sql)
df

,source_medium,conversion_count
0,google/organic,70502
1,(direct)/(none),47950
2,<Other>/<Other>,33959
3,<Other>/referral,19556
4,shop.googlemerchandisestore.com/referral,13804
5,google/cpc,10224
6,(data deleted)/(data deleted),6902
7,<Other>/organic,6059
8,<Other>/(data deleted),89


### 총수익

In [13]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
)

SELECT
  CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
  ROUND(SUM(COALESCE((SELECT value.int_value FROM UNNEST(event_params) WHERE key = "value"),0) + COALESCE((SELECT value.float_value FROM UNNEST(event_params) WHERE key = "value"),0) + COALESCE((SELECT value.double_value FROM UNNEST(event_params) WHERE key = "value"),0)),2) AS value_sum
FROM user
WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
GROUP BY source_medium
ORDER BY value_sum DESC
"""

df = sql_to_dataframe(sql)
df

,source_medium,value_sum
0,google/organic,27692.27
1,(direct)/(none),20680.92
2,<Other>/<Other>,13251.85
3,<Other>/referral,11056.97
4,shop.googlemerchandisestore.com/referral,10761.37
5,(data deleted)/(data deleted),9044.73
6,google/cpc,3005.84
7,<Other>/organic,1983.70
8,<Other>/(data deleted),71.60


### 보고서

In [14]:
sql = """
WITH user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
),
new_user AS(
  SELECT
    *
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _table_suffix BETWEEN "20201110" AND "20201206"
  AND event_name = "first_visit"
),

event_user AS(
  SELECT
    CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
    COUNT(event_name) AS conversion_count
  FROM user
  WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
  AND event_name IN("purchase","begin_checkout","first_visit","predict_ltv_payer","add_to_cart","view_item","view_cart","qualified_visit","Membership","predicted_top_spenders")
  GROUP BY source_medium
  ORDER BY conversion_count DESC
)

SELECT
  user_engaged_session.source_medium,
  * EXCEPT(source_medium)
FROM(
  SELECT
    engaged_session.source_medium,
    session.new_user_count,
    engaged_session.engaged_session_count,
    session.session_count,
    ROUND(engaged_session.engaged_session_count / session.session_count * 100,2) AS engagement_ratio,
    ROUND(engaged_session.engaged_session_count / session.new_user_count,2) AS user_per_engagement_count,
    session.engagement_time,
    session.event_count,
    session.value_sum
  FROM(
    SELECT
      CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
      COUNT(DISTINCT user_pseudo_id) AS new_user_count,
      COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS session_count,
      FLOOR(SUM(FLOOR((SELECT value.int_value FROM UNNEST(event_params) WHERE key = "engagement_time_msec") / 1000)) / COUNT(DISTINCT user_pseudo_id)) AS engagement_time,
      COUNT(event_name) AS event_count,
      ROUND(SUM(COALESCE((SELECT value.int_value FROM UNNEST(event_params) WHERE key = "value"),0) + COALESCE((SELECT value.float_value FROM UNNEST(event_params) WHERE key = "value"),0) + COALESCE((SELECT value.double_value FROM UNNEST(event_params) WHERE key = "value"),0)),2) AS value_sum
    FROM user
    WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
    GROUP BY source_medium
  ) session
  INNER JOIN(
    SELECT
      CONCAT(traffic_source.source,"/",traffic_source.medium) AS source_medium,
      COUNT(DISTINCT CONCAT(user_pseudo_id, (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id"))) AS engaged_session_count
    FROM user
    WHERE EXISTS(SELECT * FROM new_user WHERE user.user_pseudo_id = new_user.user_pseudo_id)
    AND (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "session_engaged") = 1
    GROUP BY source_medium
  ) engaged_session
  ON session.source_medium = engaged_session.source_medium
  ) user_engaged_session
INNER JOIN event_user
ON user_engaged_session.source_medium = event_user.source_medium"""

df = sql_to_dataframe(sql)
df

,source_medium,new_user_count,engaged_session_count,session_count,engagement_ratio,user_per_engagement_count,engagement_time,event_count,value_sum,conversion_count
0,(direct)/(none),19504,18427,21065,87.48,0.94,83.0,275795,20680.92,47950
1,google/organic,26937,26401,29006,91.02,0.98,84.0,396733,27692.27,70502
2,<Other>/<Other>,13380,12991,13648,95.19,0.97,81.0,186686,13251.85,33959
3,(data deleted)/(data deleted),3856,1951,4533,43.04,0.51,110.0,52113,9044.73,6902
4,shop.googlemerchandisestore.com/referral,6256,4851,6742,71.95,0.78,84.0,86619,10761.37,13804
5,google/cpc,4106,3958,4134,95.74,0.96,78.0,55833,3005.84,10224
6,<Other>/referral,8197,7351,8554,85.94,0.90,84.0,114575,11056.97,19556
7,<Other>/organic,2560,2386,2573,92.73,0.93,77.0,33801,1983.70,6059
8,<Other>/(data deleted),77,34,79,43.04,0.44,82.0,967,71.60,89
